In [ ]:
import os
import json
import numpy as np
import torch
import torch.nn as nn
from torch.optim import Adam
from pymatgen.core.structure import Structure
import matplotlib.pyplot as plt
import csv
import random
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


In [ ]:
config = {
    "data_dir": "./structure",         
    "epochs": 200,                  
    "batch_size": 32,             
    "learning_rate": 0.001,      
    "radius": 8.0,                       
    "show_progress": True          
}

# random state
torch.manual_seed(52)
np.random.seed(52)


In [ ]:
class GaussianDistance:
    def __init__(self, dmin, dmax, step, var=None):
        self.filter = np.arange(dmin, dmax + step, step)
        self.var = var if var else step

    def expand(self, distances):
        return np.exp(-(distances[..., np.newaxis] - self.filter)**2 / self.var**2)
    

In [ ]:
class POSCARDataset(Dataset):
    def __init__(self, data_dir, max_nbr=12, radius=8.0):
        self.data_dir = data_dir
        self.max_nbr = max_nbr
        self.radius = radius
        self.gdf = GaussianDistance(dmin=0, dmax=radius, step=0.2)
        print(f"Number of Gaussian filters: {len(self.gdf.filter)}")      
 
        with open(os.path.join(data_dir, 'atom_init.json')) as f:
            elem_embed = json.load(f)
        self.elem_dict = {int(k): np.array(v, dtype=np.float32) for k, v in elem_embed.items()}
        

        with open(os.path.join(data_dir, 'id_prop.csv')) as f:
            self.materials = [line.strip().split(',') for line in f]


    def __len__(self):
        return len(self.materials)

    @functools.lru_cache(maxsize=None)
    def __getitem__(self, idx):
        material_id, target = self.materials[idx]
        struct_path = os.path.join(self.data_dir, material_id)
        
        crystal = Structure.from_file(struct_path)
        
        atom_features = [self.elem_dict[site.specie.number] for site in crystal]
        
        all_nbrs = crystal.get_all_neighbors(self.radius, include_index=True)
        nbr_features = []
        nbr_indices = []
        
        for nbr_list in all_nbrs:
            sorted_nbrs = sorted(nbr_list, key=lambda x: x[1])[:self.max_nbr]
            if len(sorted_nbrs) < self.max_nbr:             
                pad_num = self.max_nbr - len(sorted_nbrs)
                distances = [nbr[1] for nbr in sorted_nbrs] + [self.radius+1]*pad_num
                indices = [nbr[2] for nbr in sorted_nbrs] + [0]*pad_num
            else:
                distances = [nbr[1] for nbr in sorted_nbrs]
                indices = [nbr[2] for nbr in sorted_nbrs]
                
            nbr_features.append(distances)
            nbr_indices.append(indices)
        
        return (
            torch.FloatTensor(atom_features),
            torch.FloatTensor(self.gdf.expand(np.array(nbr_features))),
            torch.LongTensor(nbr_indices)
        ), torch.FloatTensor([float(target)]), material_id

dataset = POSCARDataset(config["data_dir"], radius=config["radius"])
print(f"Dataset loading completed，{len(dataset)}samples")


In [ ]:
def collate_pool(batch):
    atom_fea_list, nbr_fea_list, nbr_idx_list = [], [], []
    batch_target = []
    crystal_atom_idx = []
    batch_ids = []
    base_idx = 0

    for (atom_fea, nbr_fea, nbr_idx), target, cif_id in batch:
        n_i = atom_fea.shape[0]  
        atom_fea_list.append(atom_fea)
        nbr_fea_list.append(nbr_fea)
        nbr_idx_list.append(nbr_idx + base_idx)
        crystal_atom_idx.append(torch.arange(base_idx, base_idx + n_i))
        batch_target.append(target)
        batch_ids.append(cif_id)
        base_idx += n_i

    return (torch.cat(atom_fea_list, dim=0),
            torch.cat(nbr_fea_list, dim=0),
            torch.cat(nbr_idx_list, dim=0),
            crystal_atom_idx), \
           torch.stack(batch_target, dim=0), \
           batch_ids

def split_dataset(dataset, ratios=(0.8, 0.1, 0.1)):
    indices = np.random.permutation(len(dataset))
    train_end = int(ratios[0]*len(dataset))
    val_end = train_end + int(ratios[1]*len(dataset))
    
    return {
        "train": DataLoader(dataset, batch_size=config["batch_size"],
                          sampler=SubsetRandomSampler(indices[:train_end]),
                          collate_fn=collate_pool),
        "val": DataLoader(dataset, batch_size=config["batch_size"],
                        sampler=SubsetRandomSampler(indices[train_end:val_end]),
                        collate_fn=collate_pool),
        "test": DataLoader(dataset, batch_size=config["batch_size"],
                         sampler=SubsetRandomSampler(indices[val_end:]),
                         collate_fn=collate_pool)
    }

loaders = split_dataset(dataset)
print("Data loader successfully created.")
print(f"Train set batch: {len(loaders['train'])} | Validation set batch: {len(loaders['val'])} | Test set batch: {len(loaders['test'])}")


In [ ]:
# Take a sample of data
sample_batch = next(iter(loaders['train']))
(atom_fea, nbr_fea, nbr_idx, crystal_idx), target, cif_id = sample_batch


In [ ]:
class CGCNN(nn.Module):
    def __init__(self, orig_dim, nbr_dim, hidden_dim=64):
        super().__init__()
       
        self.atom_embed = nn.Sequential(
            nn.Linear(orig_dim, hidden_dim),
            nn.Softplus()
        )

        self.conv_layers = nn.ModuleList([
            nn.Sequential(
                nn.Linear(2*hidden_dim + nbr_dim, 2*hidden_dim),
                nn.BatchNorm1d(2*hidden_dim),
                nn.Sigmoid(),
                nn.Linear(2*hidden_dim, hidden_dim),
                nn.BatchNorm1d(hidden_dim),
                nn.Softplus()
            ) for _ in range(config["n_conv_layers"])
        ])
        
        self.regressor = nn.Sequential(
            nn.Linear(hidden_dim, 128),
            nn.Softplus(),
            nn.Linear(128, 1)
        )

    def forward(self, atom_fea, nbr_fea, nbr_idx, crystal_idx):
        x = self.atom_embed(atom_fea)        
        for conv in self.conv_layers:
            N, M = nbr_idx.shape
            orig_feat = x.unsqueeze(1).expand(N, M, -1)
            nbr_feat = x[nbr_idx.view(-1)].view(N, M, -1)
            total_feat = torch.cat([orig_feat, nbr_feat, nbr_fea], dim=2)
            delta = conv(total_feat.view(-1, total_feat.shape[-1])).view(N, M, -1)
            x = x + torch.sum(delta, dim=1)
        
        crystal_feat = torch.stack([torch.mean(x[idx], dim=0) for idx in crystal_idx])
        return self.regressor(crystal_feat)


In [ ]:
model = CGCNN(
    orig_dim=atom_fea.shape[-1],
    nbr_dim=nbr_fea.shape[-1],
    hidden_dim=64
)


In [ ]:
import json
import matplotlib.pyplot as plt
import os
import torch
from torch.optim import Adam
import torch.nn as nn

def train_model():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    optimizer = Adam(model.parameters(), lr=config["learning_rate"])
    criterion = nn.MSELoss()
    
    history = {
        "train": [],
        "val": [],
        "config": config 
    }
    
    for epoch in range(1, config["epochs"]+1):

        model.train()
        train_loss = 0.0
        for batch in loaders["train"]:
            (atom_fea, nbr_fea, nbr_idx, crystal_idx), target, _ = batch
            atom_fea = atom_fea.to(device)
            nbr_fea = nbr_fea.to(device)
            nbr_idx = nbr_idx.to(device)            
            optimizer.zero_grad()
            pred = model(atom_fea, nbr_fea, nbr_idx, crystal_idx)
            loss = criterion(pred, target.to(device))
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * len(target)
        

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for batch in loaders["val"]:
                (atom_fea, nbr_fea, nbr_idx, crystal_idx), target, _ = batch
                pred = model(
                    atom_fea.to(device),
                    nbr_fea.to(device),
                    nbr_idx.to(device),
                    crystal_idx
                )
                val_loss += criterion(pred, target.to(device)).item() * len(target)

        avg_train = train_loss / len(loaders["train"].sampler)
        avg_val = val_loss / len(loaders["val"].sampler)
        history["train"].append(avg_train)
        history["val"].append(avg_val)

        if config["show_progress"]:
            print(f"Epoch {epoch:03d}/{config['epochs']} | "
                  f"Train Loss: {avg_train:.4f} | Val Loss: {avg_val:.4f}")

    visualize_history(history)

    save_history(history, "training_history.json")
    
    return model, history

def save_history(history, filename):

    dir_path = os.path.dirname(filename)
    
    if dir_path and not os.path.exists(dir_path):
        os.makedirs(dir_path, exist_ok=True)


    serializable_history = {
        "train": history["train"],
        "val": history["val"],
        "config": history["config"]
    }
    
    with open(filename, 'w') as f:
        json.dump(serializable_history, f, indent=4)
    
    print(f"Training history has been saved to{filename}")


# Training
trained_model, training_history = train_model()

# Saving historical records
save_history(training_history, "training_history.json")



In [ ]:
def evaluate_model(model, loader, device):
    model.eval()
    all_targets = []
    all_preds = []
    
    with torch.no_grad():
        for batch in loader:
            (atom_fea, nbr_fea, nbr_idx, crystal_idx), target, _ = batch
            pred = model(
                atom_fea.to(device),
                nbr_fea.to(device),
                nbr_idx.to(device),
                crystal_idx
            )
            all_targets.append(target.numpy())
            all_preds.append(pred.cpu().numpy())
    
    targets = np.concatenate(all_targets, axis=0)
    preds = np.concatenate(all_preds, axis=0)
    
    return {
        "mse": mean_squared_error(targets, preds),
        "mae": mean_absolute_error(targets, preds),
        "r2": r2_score(targets, preds),
        "targets": targets,
        "preds": preds
    }

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_metrics = evaluate_model(trained_model, loaders["train"], device)
val_metrics = evaluate_model(trained_model, loaders["val"], device)  
test_metrics = evaluate_model(trained_model, loaders["test"], device)

print(f"{'Metric':<10} | {'Training Set':>12} | {'Test Set':>10}")
print("-" * 40)
for metric in ["mse", "mae", "r2"]:
    print(f"{metric.upper():<10} | {train_metrics[metric]:>12.4f} | {test_metrics[metric]:>10.4f}")
    

In [ ]:
torch.save(trained_model.state_dict(), 'pretrained model.pth')
